# Download TCGA-BRCA tabular data

Using the manifest file downloaded of TCGA GDC for bcr-xml files of the TCGA-BRCA project is downloaded the clinical and biospecimen suplementary information for each case.

packages:

In [1]:
import tempfile
import os
import pandas as pd
from bs4 import BeautifulSoup
import pandas as pd

Important paths:

In [3]:
#Where the manifest fiel of TCGA-GDC is stored"
ManifestFilePath = '/run/media/jmalagont/ExternalDisk/Thesis/BRCA-WSI-surv/Datasets/src/tabular data manifest.txt'

#Where to download the original XML data 
XMLdstpath = '/run/media/jmalagont/ExternalDisk/Thesis/BRCA-WSI-surv/Datasets/src/'

#Where to store the preprocesed tabular data 
CSVdstpath = '/run/media/jmalagont/ExternalDisk/Thesis/BRCA-WSI-surv/Datasets/tabular/'
ManifestFilePath = '/run/media/jmalagont/ExternalDisk/Thesis/BRCA-WSI-surv/Datasets/src/tabular data manifest.txt'
dstpath = '/run/media/jmalagont/ExternalDisk/Thesis/BRCA-WSI-surv/Datasets/src/'

# 1. Download XML files

In [4]:
#Read manifest file
ManifestFile = pd.read_csv(ManifestFilePath, sep = '\t')
ManifestFile.head()
ManifestFile.head()

,id,filename,md5,size,state
0,5a13d354-608f-472e-a7c1-3b8ef0a7f28e,nationwidechildrens.org_biospecimen.TCGA-AO-A1...,3a84995c5a21ee1706f68aff9412f2cc,72693,validated
1,ef135198-15fc-4f80-86f5-a4a40a4dbe3a,nationwidechildrens.org_biospecimen.TCGA-E2-A1...,e73b359983fa1f37d7f70a6541b4a4f7,64244,validated
2,f635e122-e061-44fc-956c-9697324a0744,nationwidechildrens.org_biospecimen.TCGA-E2-A1...,bc0c8ced340b194d14160a72687610e0,64244,validated
3,c4546bf1-f47b-4fa4-945b-6203c823e13c,nationwidechildrens.org_biospecimen.TCGA-A8-A0...,4db521168cf6abdb69acb594f03e16a5,94046,validated
4,5c79e053-076f-4c77-8a15-da0dde63ab14,nationwidechildrens.org_biospecimen.TCGA-A8-A0...,4379b8bb9434177a0c27c00242eba04d,90916,validated


In [5]:
#Extract file id
files_ids = list(ManifestFile['id'])

#Transform the ids to use it in GCD API\n",
#Example available in https://docs.gdc.cancer.gov/API/Users_Guide/Downloading_Files/#downloading-multiple-files-using-post
payload_content = [f'ids={file_id}' for file_id in files_ids]
payload_content = '&'.join(payload_content)

#Store the payload in a temporal file 
TempFile = tempfile.NamedTemporaryFile()
TempFile.write(payload_content.encode())
TempFile.seek(0)

#Build a bash comand to download and clean the data using curl\n",
BashCommand = f"""
                cd '{XMLdstpath}' &&
                cd '{dstpath}' &&
                curl --remote-name --remote-header-name --request POST 'https://api.gdc.cancer.gov/data' --data @{TempFile.name} &&
                tar -xf *.tar.gz &&
                cp */* . &&
                rm *.tar.gz MANIFEST.txt &&
                rm -r */ &&
                mkdir bioespecimen clinical &&
                mv nationwidechildrens.org_biospecimen* ./bioespecimen &&
                mv nationwidechildrens.org_clinical* ./clinical 
                """
os.system(BashCommand)

TempFile.close()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.7M    0 11.6M  100 89994  39664    292  0:05:08  0:05:07  0:00:01     0


# 2. Transform XML to tabular form

In [6]:
# Tabular data frames
ClinicalData = pd.DataFrame()
BioespecimenData = pd.DataFrame()

## 2.1 Transform clinical data

In [7]:
#Get files name
files_name = [file_name for file_name in os.listdir(f'{XMLdstpath}clinical') if os.path.isfile(f'{XMLdstpath}clinical/{file_name}')] 

for file_name in files_name:

    with open(f'{XMLdstpath}clinical/{file_name}', 'r') as file:
        # Read XML file
        data = file.read()
        # Transofrm the string to BF soup
        soup = BeautifulSoup(data, "xml")
        #Extract all the TAGs and its content
        tabular_data = {tag.name: tag.text for tag in soup.find_all()}
        #Add the information to a dataframe
        ClinicalData = pd.concat([ClinicalData, pd.DataFrame(tabular_data, index=[0])], ignore_index=True)
        
        print(f'Readed {len(ClinicalData)} file of {len(files_name)}', end = '\r')

ClinicalData.head()

,tcga_bcr,admin,bcr,file_uuid,batch_number,project_code,disease_code,day_of_dcc_upload,month_of_dcc_upload,year_of_dcc_upload,...,radiation_type_notes,radiation_dosage,units,numfractions,anatomic_treatment_site,radiation_treatment_ongoing,course_number,days_to_additional_surgery_locoregional_procedure,additional_surgery_metastatic_procedure,additional_study
0,\n\nNationwide Children's Hospital\nB15D0899-2...,\nNationwide Children's Hospital\nB15D0899-286...,Nationwide Children's Hospital,B15D0899-286F-4738-B02C-C9761CABCD8D,96.80.0,TCGA,BRCA,22,12,2016,...,,6600,cGy,,Primary Tumor Field,NO,1,NaN,NaN,NaN
1,\n\nNationwide Children's Hospital\n353057F6-3...,\nNationwide Children's Hospital\n353057F6-3A0...,Nationwide Children's Hospital,353057F6-3A05-48AC-BDC7-FF3AE0A7F6F2,61.89.0,TCGA,BRCA,22,12,2016,...,,6040,cGy,33,Primary Tumor Field,NO,1,NaN,NaN,NaN
2,\n\nNationwide Children's Hospital\nB89F92DB-4...,\nNationwide Children's Hospital\nB89F92DB-4CC...,Nationwide Children's Hospital,B89F92DB-4CC1-4CA0-AF9E-09F6D68F9DC9,61.89.0,TCGA,BRCA,22,12,2016,...,,15120,cGy,28,Regional site,NO,1,,,NaN
3,\n\nNationwide Children's Hospital\nDB985E08-3...,\nNationwide Children's Hospital\nDB985E08-32B...,Nationwide Children's Hospital,DB985E08-32B2-4AE8-849F-721889157674,109.89.0,TCGA,BRCA,22,12,2016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,\n\nNationwide Children's Hospital\nF41342AB-E...,\nNationwide Children's Hospital\nF41342AB-EDF...,Nationwide Children's Hospital,F41342AB-EDF0-46DC-81B4-03A05EEEC354,117.79.0,TCGA,BRCA,22,12,2016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2.2 Transform bioespecimen data

In [9]:
#Get files name
files_name = [file_name for file_name in os.listdir(f'{XMLdstpath}bioespecimen') if os.path.isfile(f'{XMLdstpath}bioespecimen/{file_name}')] 

for file_name in files_name:
    
    with open(f'{XMLdstpath}bioespecimen/{file_name}', 'r') as file:
        # Read XML file
        data = file.read()
        # Transofrm the string to BF soup
        soup = BeautifulSoup(data, "xml")
        #Extract all the TAGs and its content
        tabular_data = {tag.name: tag.text for tag in soup.find_all()}
        #Add the information to a dataframe
        BioespecimenData = pd.concat([BioespecimenData, pd.DataFrame(tabular_data, index=[0])], ignore_index=True)
        
        print(f'Readed {len(BioespecimenData)} file of {len(files_name)}', end = '\r')

BioespecimenData.head()

,tcga_bcr,admin,bcr,file_uuid,batch_number,project_code,disease_code,day_of_dcc_upload,month_of_dcc_upload,year_of_dcc_upload,...,shipment_portion,shipment_portion_day_of_shipment,shipment_portion_month_of_shipment,shipment_portion_year_of_shipment,shipment_portion_bcr_aliquot_barcode,bcr_shipment_portion_uuid,bcr_patient_canonical_reasons,bcr_canonical_reason,additional_study,bcr_biospecimen_canonical_reasons
0,\n\nNationwide Children's Hospital\n14E062E6-1...,\nNationwide Children's Hospital\n14E062E6-106...,Nationwide Children's Hospital,14E062E6-1064-4E90-B930-ACB595A12379,360.43.0,TCGA,BRCA,1,7,2016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,\n\nNationwide Children's Hospital\nE341A5B2-0...,\nNationwide Children's Hospital\nE341A5B2-028...,Nationwide Children's Hospital,E341A5B2-0282-4656-B8AA-13F721D17C14,85.87.0,TCGA,BRCA,18,5,2022,...,\n\n21\n2\nA13E\n20\n22\n3\n2011\nTCGA-E2-A159...,22,3,2011,TCGA-E2-A159-01A-21-A13E-20,c9662a04-c4e9-42c3-989f-cade253340dc,NaN,NaN,NaN,NaN
2,\n\nNationwide Children's Hospital\n6C5FF554-D...,\nNationwide Children's Hospital\n6C5FF554-DDC...,Nationwide Children's Hospital,6C5FF554-DDC1-48D0-81D6-6177200BC840,124.81.0,TCGA,BRCA,1,7,2016,...,\n\n21\n2\nA17K\n20\n2\n8\n2011\nTCGA-D8-A1XR-...,2,8,2011,TCGA-D8-A1XR-01A-21-A17K-20,F97F91F5-8D8B-494D-847C-848251489758,NaN,NaN,NaN,NaN
3,\n\nNationwide Children's Hospital\n533C4425-1...,\nNationwide Children's Hospital\n533C4425-121...,Nationwide Children's Hospital,533C4425-1215-4924-81A6-B393BA1C1CB5,120.77.0,TCGA,BRCA,1,7,2016,...,\n\n21\n2\nA17K\n20\n2\n8\n2011\nTCGA-D8-A1XD-...,2,8,2011,TCGA-D8-A1XD-01A-21-A17K-20,71A3DDCC-C443-445F-B3C6-E0D36183AC53,NaN,NaN,NaN,NaN
4,\n\nNationwide Children's Hospital\n70CBC38B-B...,\nNationwide Children's Hospital\n70CBC38B-B37...,Nationwide Children's Hospital,70CBC38B-B37C-4450-AF9F-3DEBF58EB5AD,109.89.0,TCGA,BRCA,1,7,2016,...,\n\n21\n2\nA17J\n20\n2\n8\n2011\nTCGA-D8-A1J9-...,2,8,2011,TCGA-D8-A1J9-01A-21-A17J-20,07D51184-85A3-4BA7-9CC2-BADCB3CCFA23,NaN,NaN,NaN,NaN


## 2.3 Save data frames

In [10]:
ClinicalData.to_csv(f'{CSVdstpath}clinical_data.csv', index= False)
BioespecimenData.to_csv(f'{CSVdstpath}bioespecimen_data.csv', index = False)